In [18]:
import pandas as pd
import os
import torch
import joblib
from sklearn.preprocessing import StandardScaler  # If used during training
from ModelTrainer import ModelTrainer  # If available
from params import MODELS_DIR


# Define the directory where models are saved
input_size_model_path = os.path.join(MODELS_DIR, 'model_RandomForest_Function Input.pkl')  # Example model for Input Size
cpu_model_path = os.path.join(MODELS_DIR, 'model_RandomForest_Max CPU Usage.pkl')  # Example model for CPU
mem_model_path = os.path.join(MODELS_DIR, 'model_RandomForest_Max Memory Usage.pkl')  # Example model for Memory


cpu_model = joblib.load(cpu_model_path)
mem_model = joblib.load(mem_model_path)
size_model = joblib.load(input_size_model_path)

# Load new data
data_path = './mytest.csv'
new_data = pd.read_csv(data_path)

features = ['DAG Input Size', 'Function Name']
target = 'Function Input'

X_new = ModelTrainer(None).prepare_inference_data(new_data, features)

new_data['Predicted Input Size'] = size_model.predict(X_new)
X_new['Predicted Input Size'] = new_data['Predicted Input Size'].values
    

# Assuming 'ModelTrainer' or similar utility for data preparation is available
# Prepare data (ensure the feature names and model input match training setup)
features = ['Function Name', 'Predicted Input Size']  # Example features used for training
target = 'Max CPU Usage'  # Example target used for training
X_new = ModelTrainer(None).prepare_inference_data(new_data, features)

# X_new = new_data[features]



In [19]:
#change column name from  Predicted Input Size to Function Input
X_new = X_new.rename(columns = {'Predicted Input Size':'Function Input'})
new_data['Predicted Max CPU Usage'] = cpu_model.predict(X_new)
new_data['Predicted Max Memory Usage'] = mem_model.predict(X_new)
new_data.to_csv('./updated_new_data.csv', index=False)  # Save to CSV
print(new_data.head())  # Display the first few rows of the updated DataFrame

                             Unique DAG ID  DAG Input Content  DAG Input Size  \
0  AS-11e89baf-979f-4b15-9337-485987d59503                 50           10200   
1  AS-9f122354-8cb0-419e-9bd1-e6520454e430                 50            4200   
2  AS-49e2b177-2137-4541-a2d6-7629af7284bd                 50            6300   
3  AS-49e2b177-2137-4541-a2d6-7629af7284bd                 50            6300   
4  AS-2ec208de-f336-4341-999b-3e7f0ea1522d                 50            2400   

  Function Name  Function Input  Duration  Parallel Duration  \
0          AES1           10200      4137            12029.0   
1          AES3           37800      5064             5064.0   
2         wait1            6300      1005                NaN   
3          AES1            6300      2439             7600.0   
4         Stats            2400        25                NaN   

   Memory_Allocated  CPU_Allocated  Max Memory Usage  Max CPU Usage  \
0               256              8         21.536768     

In [9]:
# # Assuming 'ModelTrainer' or similar utility for data preparation is available
# # Prepare data (ensure the feature names and model input match training setup)
# features = ['Function Name', 'Predicted Input Size']  # Example features used for training
# target = 'Max CPU Usage'  # Example target used for training
# X_new = ModelTrainer(None).prepare_inference_data(new_data, features)

# # X_new = new_data[features]


# # Make predictions
# new_data['Predicted Max CPU Usage'] = cpu_model.predict(X_new)
# new_data['Predicted Max Memory Usage'] = mem_model.predict(X_new)

# # Save or display the updated DataFrame
# new_data.to_csv('./updated_new_data.csv', index=False)  # Save to CSV
# print(new_data.head())  # Display the first few rows of the updated DataFrame


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Predicted Input Size
Feature names seen at fit time, yet now missing:
- Function Input


In [20]:
from ModelEvalNew import ensure_1d_array
y_test = new_data['Max CPU Usage']
y_pred = new_data['Predicted Max CPU Usage']
cpu_err, cpu_err_list = ModelTrainer.calculate_cpu_usage_error_rate(ensure_1d_array(y_test), ensure_1d_array(y_pred))
cpu_err

0.1188118811881188

In [21]:
from ModelEvalNew import ensure_1d_array
y_test = new_data['Max Memory Usage']
y_pred = new_data['Predicted Max Memory Usage']
mem_err, mem_err_list = ModelTrainer.calculate_memory_usage_error_rate(ensure_1d_array(y_test), ensure_1d_array(y_pred))
mem_err

0.0

In [25]:
names = ['wait1', 'AES1', 'AES2', 'AES3', 'Stats']
DAG_input_size = [5000, 5000, 5000, 5000, 5000] 
# Function_input = [0, 0, 0, 0, 0]
dict = {'Function Name': names, 'DAG Input Size': DAG_input_size, 'Function Input': Function_input}
request = pd.DataFrame(dict)

features = ['DAG Input Size', 'Function Name']
target = 'Function Input'


X_new = ModelTrainer(None).prepare_inference_data(request, features)

request['Predicted Input Size'] = size_model.predict(X_new)
X_new['Predicted Input Size'] = request['Predicted Input Size'].values
    

# Assuming 'ModelTrainer' or similar utility for data preparation is available
# Prepare data (ensure the feature names and model input match training setup)
features = ['Function Name', 'Predicted Input Size']  # Example features used for training
target = 'Max CPU Usage'  # Example target used for training
X_new = ModelTrainer(None).prepare_inference_data(request, features)

#change column name from  Predicted Input Size to Function Input
X_new = X_new.rename(columns = {'Predicted Input Size':'Function Input'})
request['Predicted Max CPU Usage'] = cpu_model.predict(X_new)
request['Predicted Max Memory Usage'] = mem_model.predict(X_new)



In [26]:
request

,Function Name,DAG Input Size,Function Input,Predicted Input Size,Predicted Max CPU Usage,Predicted Max Memory Usage
0,wait1,5000,0,4996.0,0.015254,10.003743
1,AES1,5000,0,4998.0,0.655869,22.907126
2,AES2,5000,0,19960.0,1.307078,28.286362
3,AES3,5000,0,44649.0,1.971335,34.219090
4,Stats,5000,0,4996.0,0.004952,10.984899
